# Neural Network

### Feature Engineering and Feature Selection

In [1]:
#import all the required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

/Users/manojkashyap/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
import tensorflow

In [3]:
tensorflow.__version__

'2.4.1'

In [4]:
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch

In [5]:
order = pd.read_csv('olist_order_items_dataset.csv') #orders order id PK

In [6]:
order.head(2)

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.9,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.9,19.93


In [7]:
d_set = order
d_set.head(2)

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.9,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.9,19.93


In [8]:
d_set = d_set.sort_values('shipping_limit_date')
d_set.head(2)

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
101394,e5fa5a7210941f7d56d0208e4e071d35,1,f3c2d01a84c947b078e32bbef0718962,a425f92c199eb576938df686728acd20,2016-09-19 00:15:34,59.50,15.56
84389,bfbd0f9bdef84302105ad712db648a6c,1,5a6b04657a4c5ee34285d1e4619a96b4,ecccfa2bb93b34a3bf033cc5d1dcdc69,2016-09-19 23:11:33,44.99,2.83


In [9]:
d_set = d_set[['shipping_limit_date', 'price', 'freight_value']]
d_set.head(2)

,shipping_limit_date,price,freight_value
101394,2016-09-19 00:15:34,59.50,15.56
84389,2016-09-19 23:11:33,44.99,2.83


In [10]:
d_set.dtypes

shipping_limit_date     object
price                  float64
freight_value          float64
dtype: object

In [11]:
d_set['shipping_limit_date'] = pd.to_datetime(d_set['shipping_limit_date'])
d_set.dtypes

shipping_limit_date    datetime64[ns]
price                         float64
freight_value                 float64
dtype: object

In [12]:
#https://stackoverflow.com/questions/26646191/pandas-groupby-month-and-year
d_set.set_index('shipping_limit_date', inplace=True)
d_set = d_set.resample("D").sum()

In [13]:
d_set

,price,freight_value
shipping_limit_date,,
2016-09-19,194.47,24.05
2016-09-20,0.00,0.00
2016-09-21,0.00,0.00
2016-09-22,0.00,0.00
2016-09-23,0.00,0.00
...,...,...
2020-04-05,0.00,0.00
2020-04-06,0.00,0.00
2020-04-07,0.00,0.00


In [14]:
d_set.shape

(1299, 2)

> Droping outliers

In [15]:
#https://www.geeksforgeeks.org/how-to-drop-rows-in-dataframe-by-conditions-on-column-values/
d_set = d_set.drop(d_set[d_set['price'] == 0.00].index, axis= 0)

In [16]:
d_set

,price,freight_value
shipping_limit_date,,
2016-09-19,194.47,24.05
2016-10-08,908.29,194.09
2016-10-09,1545.57,243.37
2016-10-10,7607.91,1323.12
2016-10-11,7350.06,1043.57
...,...,...
2018-09-14,619.99,48.43
2018-09-18,999.99,24.77
2020-02-03,75.99,14.70


In [17]:
d_set.rename(columns= {'price': 'Total sales'}, inplace= True)
d_set.index.names = ['Daily Shipping']

In [18]:
d_set['Seasonal First Difference'] = d_set['Total sales'] - d_set['Total sales'].shift(1)

In [19]:
# Testing whether the data is Stationary or not
from statsmodels.tsa.stattools import adfuller

/Users/manojkashyap/anaconda3/lib/python3.7/site-packages/statsmodels/compat/pandas.py:23: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  data_klasses = (pandas.Series, pandas.DataFrame, pandas.Panel)


In [20]:
test_result= adfuller(d_set['Total sales'])

In [21]:
#https://github.com/krishnaik06/ARIMA-And-Seasonal-ARIMA/blob/master/Untitled.ipynb
#Ho: It is non stationary
#H1: It is stationary

def adfuller_test(sales):
    result=adfuller(sales)
    labels = ['ADF Test Statistic','p-value','#Lags Used','Number of Observations Used']
    for value,label in zip(result,labels):
        print(label+' : '+str(value) )
    if result[1] <= 0.05:
        print("strong evidence against the null hypothesis(Ho), reject the null hypothesis. Data has no unit root and is stationary")
    else:
        print("weak evidence against null hypothesis, time series has a unit root, indicating it is non-stationary ")

In [22]:
adfuller_test(d_set['Total sales'])

ADF Test Statistic : -2.4295615100445827
p-value : 0.1335661063193953
#Lags Used : 18
Number of Observations Used : 536
weak evidence against null hypothesis, time series has a unit root, indicating it is non-stationary 


In [23]:
d_set['Seasonal First Difference'] = d_set['Total sales'] - d_set['Total sales'].shift(1)

In [24]:
## Again test dickey fuller test
adfuller_test(d_set['Seasonal First Difference'].dropna())

ADF Test Statistic : -8.08197541986415
p-value : 1.4532141790126681e-12
#Lags Used : 17
Number of Observations Used : 536
strong evidence against the null hypothesis(Ho), reject the null hypothesis. Data has no unit root and is stationary


In [25]:
X = d_set.dropna().drop(['Total sales'], axis= 1)
y = d_set.dropna().drop(['freight_value', 'Seasonal First Difference'], axis= 1)

In [26]:
#Standardizing the data
from sklearn.preprocessing import StandardScaler
sc_x = StandardScaler()
X = sc_x.fit_transform(X)
X = pd.DataFrame(X)

In [27]:
#https://www.kaggle.com/antoinekrajnc/simplest-time-series-using-linear-regression
def TimeSeriesTrainTestSplit(x, y, test_size):
    
        test_index = int(len(X)*(1-test_size))
    
        X_train = X.iloc[:test_index]
        y_train = y.iloc[:test_index]
        X_test = X.iloc[test_index:]
        y_test = y.iloc[test_index:]
        return X_train, y_train, X_test, y_test
    
X_train, y_train, X_test, y_test = TimeSeriesTrainTestSplit(X,y, 0.2)

## Neural Network using keras (With Hyperparameter tuning)

### 1. activation= relu , optimizer = Adam

In [69]:
#
def build_model(hp):
    model = keras.Sequential()
    for i in range(hp.Int('num_layers', 2, 20)): #Neura network may have layers 2 to 20
        #Creating Nodes(using layers.Dense) inside every layer...each layer may have max of 512 & min of 32 Nodes
        #hp.Int: Try with different values
        model.add(layers.Dense(units=hp.Int('units_' + str(i),
                                            min_value=32,
                                            max_value=512,
                                            step=32),
                               activation='relu')) #activation= relu
    #Adding Output layer: Dense layer (Number of Output Neurons= 1, bcz it is a Regression problem)
    #For output layer we use activation='linear'..since it is a Regression problem
    model.add(layers.Dense(1, activation='linear'))
    #Compiling
    model.compile(
        #optimizers = Adam
        optimizer=keras.optimizers.Adam(
            ##hp.Choice: Choose any one value out of given values
            hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])), #Hyper-parameters
        loss='mean_absolute_error', #Choosing MAE as loss-function
        metrics=['mean_absolute_error'])
    return model

#### Total Hyperparameters used:
1. How many number of Hidden layers we should have? :(2 to 20 given)
2. How many number of neurons we should have in hidden layers? :(32 to 512 given)
3. Lernin rate :(3 values selected-> [1e-2, 1e-3, 1e-4])

In [70]:
# https://keras-team.github.io/keras-tuner/
#Performing RandomSearch to find the best model
tuner = RandomSearch(
    build_model,
    objective='val_mean_absolute_error',
    max_trials=5,
    executions_per_trial=3,
    directory='project', #Creating Directory
    project_name='Price prediction')

> The build_model will get executed 5*3 =15 different iterations/times to see whether it is working well(MAE)

In [71]:
tuner.search_space_summary()

Search space summary
Default search space size: 4
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 20, 'step': 1, 'sampling': None}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


In [72]:
#Calling RandomSearch
tuner.search(X_train, y_train,
             epochs=50, #Run it for 50 epochs
             validation_data=(X_test, y_test)) #Giving test data as validation data

Trial 5 Complete [00h 00m 36s]
val_mean_absolute_error: 3109.2725423177085

Best val_mean_absolute_error So Far: 2817.48486328125
Total elapsed time: 00h 02m 22s
INFO:tensorflow:Oracle triggered exit


In [73]:
#Showing 10 best trials
tuner.results_summary()

Results summary
Results in project/Price prediction
Showing 10 best trials
Objective(name='val_mean_absolute_error', direction='min')
Trial summary
Hyperparameters:
num_layers: 7
units_0: 384
units_1: 128
learning_rate: 0.001
units_2: 128
units_3: 128
units_4: 448
units_5: 352
units_6: 288
units_7: 384
units_8: 224
units_9: 192
units_10: 128
units_11: 256
Score: 2817.48486328125
Trial summary
Hyperparameters:
num_layers: 16
units_0: 96
units_1: 512
learning_rate: 0.0001
units_2: 224
units_3: 224
units_4: 96
units_5: 192
units_6: 448
units_7: 192
units_8: 224
units_9: 32
units_10: 384
units_11: 128
units_12: 32
units_13: 32
units_14: 32
units_15: 32
Score: 3109.2725423177085
Trial summary
Hyperparameters:
num_layers: 12
units_0: 160
units_1: 384
learning_rate: 0.0001
units_2: 64
units_3: 352
units_4: 384
units_5: 192
units_6: 384
units_7: 480
units_8: 32
units_9: 32
units_10: 32
units_11: 32
Score: 3367.095458984375
Trial summary
Hyperparameters:
num_layers: 8
units_0: 320
units_1: 224


> Objective(name='val_mean_absolute_error', direction='min') : Means objective is to minimize-val_mean_absolute_error{MAE} (Since it is Regression)
>> In Classification instead of MAE..we can have Accuracy

* With learning_rate: 0.001, and 7 layers as specified above, we got a score(MAE) of 2817 ...This is the highest ever achieved

###  2. activation= sigmoid , optimizer = Adam

In [63]:
def build_model_1(hp):
    model = keras.Sequential()
    for i in range(hp.Int('num_layers', 2, 20)):
        model.add(layers.Dense(units=hp.Int('units_' + str(i),
                                            min_value=32,
                                            max_value=512,
                                            step=32),
                               activation='sigmoid'))
    model.add(layers.Dense(1, activation='linear'))
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
        loss='mean_absolute_error',
        metrics=['mean_absolute_error'])
    return model

In [64]:
tuner_1 = RandomSearch(
    build_model_1,
    objective='val_mean_absolute_error',
    max_trials=5,
    executions_per_trial=3,
    directory='project_1',
    project_name='Price prediction')

In [65]:
tuner_1.search_space_summary()

Search space summary
Default search space size: 4
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 20, 'step': 1, 'sampling': None}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


In [66]:
tuner_1.search(X_train, y_train,
             epochs=10,
             validation_data=(X_test, y_test))

Trial 5 Complete [00h 00m 08s]
val_mean_absolute_error: 31044.6171875

Best val_mean_absolute_error So Far: 31013.793619791668
Total elapsed time: 00h 00m 54s
INFO:tensorflow:Oracle triggered exit


In [68]:
tuner_1.results_summary()

Results summary
Results in project_1/Price prediction
Showing 10 best trials
Objective(name='val_mean_absolute_error', direction='min')
Trial summary
Hyperparameters:
num_layers: 11
units_0: 288
units_1: 320
learning_rate: 0.01
units_2: 128
units_3: 416
units_4: 96
units_5: 480
units_6: 480
units_7: 256
units_8: 32
units_9: 32
units_10: 32
Score: 31013.793619791668
Trial summary
Hyperparameters:
num_layers: 7
units_0: 64
units_1: 512
learning_rate: 0.01
units_2: 32
units_3: 32
units_4: 32
units_5: 32
units_6: 32
Score: 31014.951171875
Trial summary
Hyperparameters:
num_layers: 5
units_0: 384
units_1: 256
learning_rate: 0.0001
units_2: 384
units_3: 352
units_4: 96
units_5: 32
units_6: 32
units_7: 192
units_8: 96
units_9: 416
units_10: 416
units_11: 480
units_12: 320
units_13: 512
units_14: 64
units_15: 64
Score: 31044.6171875
Trial summary
Hyperparameters:
num_layers: 8
units_0: 64
units_1: 512
learning_rate: 0.001
units_2: 320
units_3: 480
units_4: 480
units_5: 224
units_6: 352
units_7

### 3. activation= softmax , optimizer = SGD

In [49]:
def build_model_2(hp):
    model = keras.Sequential()
    for i in range(hp.Int('num_layers', 2, 20)):
        model.add(layers.Dense(units=hp.Int('units_' + str(i),
                                            min_value=32,
                                            max_value=512,
                                            step=32),
                               activation='softmax'))
    model.add(layers.Dense(1, activation='softmax'))
    model.compile(
        optimizer=keras.optimizers.SGD(
            hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
        loss='mean_absolute_error',
        metrics=['mean_absolute_error'])
    return model

In [50]:
tuner_2 = RandomSearch(
    build_model_2,
    objective='val_mean_absolute_error',
    max_trials=5,
    executions_per_trial=3,
    directory='project_2',
    project_name='Price prediction_1')

INFO:tensorflow:Reloading Oracle from existing project project_2/Price prediction_1/oracle.json
INFO:tensorflow:Reloading Tuner from project_2/Price prediction_1/tuner0.json


In [51]:
tuner_2.search_space_summary()

Search space summary
Default search space size: 22
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 20, 'step': 1, 'sampling': None}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}
units_2 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
units_3 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
units_4 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
units_5 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
units_6 (Int)
{'default': None, 'co

In [52]:
tuner_2.search(X_train, y_train,
             epochs=10,
             validation_data=(X_test, y_test))

INFO:tensorflow:Oracle triggered exit


In [53]:
tuner_2.results_summary()

Results summary
Results in project_2/Price prediction_1
Showing 10 best trials
Objective(name='val_mean_absolute_error', direction='min')
Trial summary
Hyperparameters:
num_layers: 20
units_0: 320
units_1: 320
learning_rate: 0.001
units_2: 224
units_3: 256
units_4: 384
units_5: 160
units_6: 192
units_7: 224
units_8: 224
units_9: 32
units_10: 352
units_11: 256
units_12: 448
units_13: 64
units_14: 128
units_15: 32
units_16: 32
units_17: 32
units_18: 32
units_19: 32
Score: 31050.51953125
Trial summary
Hyperparameters:
num_layers: 2
units_0: 480
units_1: 288
learning_rate: 0.01
Score: 31050.51953125
Trial summary
Hyperparameters:
num_layers: 11
units_0: 448
units_1: 160
learning_rate: 0.01
units_2: 448
units_3: 160
units_4: 416
units_5: 416
units_6: 416
units_7: 288
units_8: 128
units_9: 128
units_10: 416
units_11: 288
units_12: 128
units_13: 320
units_14: 320
Score: 31050.51953125
Trial summary
Hyperparameters:
num_layers: 8
units_0: 160
units_1: 480
learning_rate: 0.01
units_2: 384
units

### 4. activation= tanh , optimizer = RMSprop

In [54]:
def build_model_3(hp):
    model = keras.Sequential()
    for i in range(hp.Int('num_layers', 2, 20)):
        model.add(layers.Dense(units=hp.Int('units_' + str(i),
                                            min_value=32,
                                            max_value=512,
                                            step=32),
                               activation='tanh'))
    model.add(layers.Dense(1, activation='linear'))
    model.compile(
        optimizer=keras.optimizers.RMSprop(
            hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
        loss='mean_absolute_error',
        metrics=['mean_absolute_error'])
    return model

In [55]:
tuner_3 = RandomSearch(
    build_model_2,
    objective='val_mean_absolute_error',
    max_trials=5,
    executions_per_trial=3,
    directory='project_3',
    project_name='Price prediction_1')

In [56]:
tuner_3.search_space_summary()

Search space summary
Default search space size: 4
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 20, 'step': 1, 'sampling': None}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


In [57]:
tuner_3.search(X_train, y_train,
             epochs=10,
             validation_data=(X_test, y_test))

Trial 5 Complete [00h 00m 15s]
val_mean_absolute_error: 31050.51953125

Best val_mean_absolute_error So Far: 31050.51953125
Total elapsed time: 00h 00m 54s
INFO:tensorflow:Oracle triggered exit


In [58]:
tuner_3.results_summary()

Results summary
Results in project_3/Price prediction_1
Showing 10 best trials
Objective(name='val_mean_absolute_error', direction='min')
Trial summary
Hyperparameters:
num_layers: 14
units_0: 448
units_1: 160
learning_rate: 0.001
units_2: 32
units_3: 32
units_4: 32
units_5: 32
units_6: 32
units_7: 32
units_8: 32
units_9: 32
units_10: 32
units_11: 32
units_12: 32
units_13: 32
Score: 31050.51953125
Trial summary
Hyperparameters:
num_layers: 7
units_0: 192
units_1: 128
learning_rate: 0.0001
units_2: 352
units_3: 32
units_4: 224
units_5: 384
units_6: 352
units_7: 320
units_8: 352
units_9: 224
units_10: 384
units_11: 256
units_12: 192
units_13: 192
Score: 31050.51953125
Trial summary
Hyperparameters:
num_layers: 8
units_0: 320
units_1: 256
learning_rate: 0.01
units_2: 352
units_3: 416
units_4: 64
units_5: 288
units_6: 96
units_7: 384
units_8: 448
units_9: 288
units_10: 384
units_11: 416
units_12: 352
units_13: 352
Score: 31050.51953125
Trial summary
Hyperparameters:
num_layers: 17
units_0:

* Using activation= relu & optimizer = Adam: With learning_rate: 0.001, and 7 layers as specified above, we got a score(MAE) of 2817 ...This is the highest ever achieved